# Machine Learning Pipelines

# Models as a task graph (DAG)
<img src="../part_1/img/rolson17-ml-pipeline.png">
<div style="text-align: right">Source: R. Olson et. al. (2017) "Data-driven Advice for Applying Machine Learning to Bioinformatics Problems."</div>

# Why a DAG?

### Encapsulation

Feature engineering operators, post-processing (e.g. calibration) are part of your model.

### Optimization

Feature engineering operators have hyper-parameters too. Furthermore, we want to search over graphs.

### Safety

Ensure that no information from testing can "leak" into the model fitting phase.

### Train-test-skew

Contracts between operators are more subtle than just message schemas; assumptions about the data distribution are included as well. Operators must not change after the model was fit!

# Pipelines in Scikit-learn

### [Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline)

A linear sequence of `Estimator` objects that represent a fixed set of steps, often feature extraction, normalization and classification. It implements the [Composite design pattern (GoF)](https://en.wikipedia.org/wiki/Composite_pattern).

All estimators in a pipeline, except the last one, must be transformers (i.e. must have a `transform` method).

Pipelines allow nested parameters:
```python

est = Pipeline(steps=[('scaler', StandardScaler()), ('lr', LinearRegression())])
est.set_params(scaler__with_std=False)
```

### Exercise 2.1.1

Build a `Pipeline` version of the `PolynomialRegressor` and search for the optimal `degree` of the polynomial.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%run ../part_1/utils.py

In [ ]:
X, y = generate_curve_data()
fig = plot_data(X, y, fn=true_fn)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Excercise: change this estimator to polynomial regression
est = LinearRegression()

gs = GridSearchCV(estimator=est, param_grid={}, 
                  scoring='neg_mean_squared_error', cv=3)

gs.fit(X[:, np.newaxis], y)

fig = plot_data(X, y, fn=true_fn)
plot_estimator(gs.best_estimator_, fig)

# Beyond a chain of estimators


### [FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#featureunion-composite-feature-spaces)

Combines several transformer objects into a new transformer that combines their output.
Takes a list of transformer objects, each of which is fit to the data independently. 
The transformers are applied in parallel, and the feature matrices they output are concatenated side-by-side into a wider matrix (via `np.hstack`).

```python
feature_extractors = [('poly', PolynomialFeatures()), ('pca', PCA())]
est = Pipeline(steps=[('fu_1', FeatureUnion(feature_extractors)), 
                      ('rfe', RFE(LinearRegression(), 5, step=1)), 
                      ('lr', LinearRegression())])
```

### Caching of transformers

Pipelines can do [caching](https://scikit-learn.org/stable/modules/compose.html#caching-transformers-avoid-repeated-computation) via `memory` which helps to avoid repeated computation. 

# Working with heterogenous data sources

### [ColumnTransformer](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)

Many datasets contain features of different types, say text, numeric, categorical, and dates, where each type of feature requires separate preprocessing or feature extraction steps.

This component is still experimental in 0.21.

```python
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
column_trans = ColumnTransformer(
    [('city_category', OneHotEncoder(dtype='int'),['city']),
     ('title_bow', CountVectorizer(), 'title')],
    remainder='drop')
```

# Kaggle Time!

### Dataset

[Blue Book for Bulldozers (aka Fastiron) Kaggle competition](https://www.kaggle.com/c/bluebook-for-bulldozers/data). 

Predict the auction sale price for a piece of heavy equipment to create a "blue book" for bulldozers.

Performance metric: mean absolute error (MAE).

In [ ]:
import pandas as pd
import numpy as np

url = 'https://s3.amazonaws.com/datarobot_public_datasets/fastiron-train-sample_80.csv'
df = pd.read_csv(url, parse_dates=['saledate'])
df['saledate'] = df['saledate'].astype(np.int64)
target_feature = 'SalePrice'
categorical_features = ['UsageBand', 'fiBaseModel', 'fiModelDesc', 
                        'fiModelSeries', 'ProductSize',
                        'state', 'ProductGroup', 'ProductGroupDesc','Enclosure',
                        'Enclosure_Type', 'Tire_Size',
                        ]
numeric_features = ['MachineID', 'ModelID', 'datasource', 'YearMade', 'saledate']
text_features = ['fiProductClassDesc']

y = df.pop(target_feature)
X = df

In [ ]:
y.plot.hist(50)

In [ ]:
y_log1p = np.log1p(y)
y_log1p.plot.hist(50)

# Example Pipeline

<img src="img/pipeline-example.png">


In [ ]:
import utils
from utils import ColumnGroupSelector
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

# y should be an ndarray
y_train = y_train.values
y_test = y_test.values


numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

## Categorical pipeline
# categorical_pipeline = Pipeline(steps=[...])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ## ('cat', categorical_pipeline, categorical_features),
    ])

# Append estimator to preprocessing pipeline.
# Now we have a full prediction pipeline.
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('lr', LinearRegression())])

est.fit(X_train, y_train)
print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

# Exercise 2.1.2

Implement the following pipeline:

<img src="img/exercise_2_1_2.png">

Tune hyper-parameters:
  * `RandomForestRegressor(criterion='mse')` with `['mse', 'mae']`
  * Set `RandomForestRegressor(n_estimators=10)` to avoid long runtime
  * Set `SelectFromModel`s model to `ElasticNet(alpha=0.1)` and select `max_features=100` and `threshold=None`

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet


numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99999)),
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features),
    ])

# Append estimator to preprocessing pipeline.
# Now we have a full prediction pipeline.
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', RandomForestRegressor(n_estimators=10, n_jobs=-1))])

gs = GridSearchCV(estimator=est, param_grid={'rf__criterion': ['mse', 'mae']}, 
                  scoring='neg_mean_absolute_error', cv=3, refit=True, verbose=10)

gs.fit(X_train, y_train)

print("best params: {}".format(gs.best_params_))
print("model score: %.3f" % mean_absolute_error(y_test, gs.best_estimator_.predict(X_test)))

# Exercise 2.1.3

Add a sub-pipeline to process text:

<img src="img/exercise_2_1_3.png">

Notes:
  * `RandomForestRegressor` is not doing great with wide & sparse Bag-of-Words representations; you can use a sub-model to predict the `SalePrice` just from BoW features and feed this predictions into the `RandomForestRegressor`.

In [ ]:

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99999)),
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        #('cat', categorical_pipeline, categorical_features),
        #('text', text_pipeline, text_features[0])
    ])

# Append estimator to preprocessing pipeline.
# Now we have a full prediction pipeline.
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', RandomForestRegressor(n_estimators=10, criterion='mae', n_jobs=-1))])

est.fit(X_train, y_train)

print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))